In [2]:
import pandas as pd
file_path = "./course_datas/c39_explode_to_manyrows/读者提供的数据-输入.xlsx"
df = pd.read_excel(file_path)

In [3]:
df.head()

,P/N,Description,Supplier,Supplier PN,Supplier.1,Supplier PN.1,Supplier.2,Supplier PN.2
0,302-462-326,CAP CER 0402 100pF 5% 50V,MURATA,GRM1555C1H101JA01D,YAGEO,CC0402JRNPO9BN101,GRM1555C1H101JA01J,Murata Electronics North America
1,302-462-012,CAP CER 0402 6.8pF 0.25pF 50V,AVX Corporation,04025A6R8CAT2A,KEMET,C0402C689C5GACTU,NaN,NaN
2,302-462-009,CAP CER 0402 3.9pF 0.25pF 50V,AVX Corporation,04025A3R9CAT2A,NaN,NaN,NaN,NaN


In [4]:
#提取待合并的所有列名，之后用于drop掉
merge_names = list(df.loc[:, "Supplier":].columns.values)

In [5]:
merge_names

['Supplier',
 'Supplier PN',
 'Supplier.1',
 'Supplier PN.1',
 'Supplier.2',
 'Supplier PN.2']

In [6]:
# 多列合并到一列
def merge_cols(x):
    #删除为空的列
    x = x[x.notna()]
    #使用x.values用于合并
    y = x.values
    #合并后的列表，每个元素是"Supplier" 和 "Supplier PN"对
    result = []
    # range的步长为2，目的是每两列做合并
    for idx in range(0, len(y), 2):
        # 竖线作为"Supplier"和"Supplier PN"的分隔符
        result.append(f'{y[idx]}|{y[idx+1]}')
    # 将所有的两两对，用#分割，返回一个大字符串
    return "#".join(result)

#添加新列，把待合并的所有列变成一个大字符串
df['merge'] = df.loc[:, 'Supplier':].apply(merge_cols, axis = 1)


In [7]:
df

,P/N,Description,Supplier,Supplier PN,Supplier.1,Supplier PN.1,Supplier.2,Supplier PN.2,merge
0,302-462-326,CAP CER 0402 100pF 5% 50V,MURATA,GRM1555C1H101JA01D,YAGEO,CC0402JRNPO9BN101,GRM1555C1H101JA01J,Murata Electronics North America,MURATA|GRM1555C1H101JA01D#YAGEO|CC0402JRNPO9BN...
1,302-462-012,CAP CER 0402 6.8pF 0.25pF 50V,AVX Corporation,04025A6R8CAT2A,KEMET,C0402C689C5GACTU,NaN,NaN,AVX Corporation|04025A6R8CAT2A#KEMET|C0402C689...
2,302-462-009,CAP CER 0402 3.9pF 0.25pF 50V,AVX Corporation,04025A3R9CAT2A,NaN,NaN,NaN,NaN,AVX Corporation|04025A3R9CAT2A


In [8]:
# 把不用的列删掉
df.drop(merge_names, axis = 1, inplace = True)

In [9]:
# 使用explode把一列变多行
df['merge'] = df['merge'].str.split("#")
df

,P/N,Description,merge
0,302-462-326,CAP CER 0402 100pF 5% 50V,"[MURATA|GRM1555C1H101JA01D, YAGEO|CC0402JRNPO9..."
1,302-462-012,CAP CER 0402 6.8pF 0.25pF 50V,"[AVX Corporation|04025A6R8CAT2A, KEMET|C0402C6..."
2,302-462-009,CAP CER 0402 3.9pF 0.25pF 50V,[AVX Corporation|04025A3R9CAT2A]


In [10]:
df_explode = df.explode('merge')

In [11]:
df_explode

,P/N,Description,merge
0,302-462-326,CAP CER 0402 100pF 5% 50V,MURATA|GRM1555C1H101JA01D
0,302-462-326,CAP CER 0402 100pF 5% 50V,YAGEO|CC0402JRNPO9BN101
0,302-462-326,CAP CER 0402 100pF 5% 50V,GRM1555C1H101JA01J|Murata Electronics North Am...
1,302-462-012,CAP CER 0402 6.8pF 0.25pF 50V,AVX Corporation|04025A6R8CAT2A
1,302-462-012,CAP CER 0402 6.8pF 0.25pF 50V,KEMET|C0402C689C5GACTU
2,302-462-009,CAP CER 0402 3.9pF 0.25pF 50V,AVX Corporation|04025A3R9CAT2A


In [12]:
df_explode['Supplier'] = df_explode['merge'].str.split('|').str[0]

In [13]:
df_explode['Supplier PN'] = df_explode['merge'].str.split('|').str[1]

In [14]:
df_explode

,P/N,Description,merge,Supplier,Supplier PN
0,302-462-326,CAP CER 0402 100pF 5% 50V,MURATA|GRM1555C1H101JA01D,MURATA,GRM1555C1H101JA01D
0,302-462-326,CAP CER 0402 100pF 5% 50V,YAGEO|CC0402JRNPO9BN101,YAGEO,CC0402JRNPO9BN101
0,302-462-326,CAP CER 0402 100pF 5% 50V,GRM1555C1H101JA01J|Murata Electronics North Am...,GRM1555C1H101JA01J,Murata Electronics North America
1,302-462-012,CAP CER 0402 6.8pF 0.25pF 50V,AVX Corporation|04025A6R8CAT2A,AVX Corporation,04025A6R8CAT2A
1,302-462-012,CAP CER 0402 6.8pF 0.25pF 50V,KEMET|C0402C689C5GACTU,KEMET,C0402C689C5GACTU
2,302-462-009,CAP CER 0402 3.9pF 0.25pF 50V,AVX Corporation|04025A3R9CAT2A,AVX Corporation,04025A3R9CAT2A


In [15]:
df.loc[:, 'Supplier':]

KeyError: 'Supplier'